# Проверка на атаку роутера


In [1]:
from scapy.all import *

# Загружаем PCAP-файл
packets = rdpcap(r"D:\Проекты\Дипломаня работа\DoFitN\Data\arp_A.pcapng")
arp_replies = [pkt for pkt in packets if pkt.haslayer(ARP) and pkt[ARP].op == 2]
legit_mappings = {}  # {IP: MAC}
spoofed_pairs = []   # Список подозрительных пар (IP, Fake_MAC)
for pkt in arp_replies:
    arp_layer = pkt[ARP]
    ip = arp_layer.psrc  # IP, за который выдается MAC
    mac = arp_layer.hwsrc # Указанный MAC

    # Если IP уже есть в словаре, но с другим MAC
    if ip in legit_mappings:
        if legit_mappings[ip] != mac:
            spoofed_pairs.append((ip, mac))
    else:
        legit_mappings[ip] = mac
# Находим IP, для которых есть несколько разных MAC
spoofed_ips = set()
for ip, fake_mac in spoofed_pairs:
    spoofed_ips.add(ip)
    print(f"[!] ARP-Spoofing detected: IP {ip} claims to be at {fake_mac}")

# Выводим результаты
print("\nSpoofed IP addresses:", spoofed_ips)

[!] ARP-Spoofing detected: IP 192.168.1.63 claims to be at 54:27:1e:9f:2c:b7
[!] ARP-Spoofing detected: IP 192.168.1.63 claims to be at 54:27:1e:9f:2c:b7
[!] ARP-Spoofing detected: IP 192.168.1.63 claims to be at 54:27:1e:9f:2c:b7
[!] ARP-Spoofing detected: IP 192.168.1.63 claims to be at 54:27:1e:9f:2c:b7
[!] ARP-Spoofing detected: IP 192.168.1.1 claims to be at 00:ad:24:bf:9d:52
[!] ARP-Spoofing detected: IP 192.168.1.63 claims to be at 54:27:1e:9f:2c:b7
[!] ARP-Spoofing detected: IP 192.168.1.63 claims to be at 54:27:1e:9f:2c:b7
[!] ARP-Spoofing detected: IP 192.168.1.63 claims to be at 54:27:1e:9f:2c:b7
[!] ARP-Spoofing detected: IP 192.168.1.63 claims to be at 54:27:1e:9f:2c:b7
[!] ARP-Spoofing detected: IP 192.168.1.63 claims to be at 54:27:1e:9f:2c:b7
[!] ARP-Spoofing detected: IP 192.168.1.63 claims to be at 54:27:1e:9f:2c:b7
[!] ARP-Spoofing detected: IP 192.168.1.63 claims to be at 54:27:1e:9f:2c:b7
[!] ARP-Spoofing detected: IP 192.168.1.63 claims to be at 54:27:1e:9f:2c:b7


# Загрузка pcap-файлов, извлечение признаков,работа с Dataframe


In [2]:
from scapy.all import rdpcap, ARP
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler



#Загрузка pcap files
Normal_traffic = rdpcap(r'D:\Проекты\Дипломаня работа\DoFitN\Data\arp_N.pcapng')
Anomal_traffic = rdpcap(r'D:\Проекты\Дипломаня работа\DoFitN\Data\arp_A.pcapng')

#Функция для вытаскивания признаков
def extract_pkt(arp_packets, label):
    data = []
    for pkt in arp_packets:
        data.append([
            pkt.time,  # временная метка пакета
            pkt[ARP].psrc,  # IP-адрес источника
            pkt[ARP].pdst,  # IP-адрес назначения
            pkt[ARP].hwsrc,  # MAC-адрес источника
            pkt[ARP].hwdst,  # MAC-адрес назначения
            pkt[ARP].op,  # Тип запроса ARP (1 - запрос, 2 - ответ)
            label  # метка: 0 - нормальный трафик, 1 - атака
        ])
    return data

normal_arp = extract_pkt(Normal_traffic, label=0)
anomal_arp = extract_pkt(Anomal_traffic, label= 1)

# Создание колонок
columns =(  'Time',
            'IP источника', 
            'ip назначения',
            'MAC источника',
            'MAC назначения',
            'Тип запроса',
            'Метка')
df_normal = pd.DataFrame(normal_arp,columns=columns)
df_anomal = pd.DataFrame(anomal_arp, columns=columns)
#Cоединение в одну колонку
df = pd.concat([df_normal,df_anomal])

le_ip = LabelEncoder()
le_mac = LabelEncoder()
le_ip.fit(df['IP источника'].tolist() + df['ip назначения'].tolist())
df['IP источника'] = le_ip.transform(df['IP источника'])
df['ip назначения'] = le_ip.transform(df['ip назначения'])

le_mac.fit(df['MAC источника'].tolist() + df['MAC назначения'].tolist())
df['MAC источника'] = le_mac.transform(df['MAC источника'])
df['MAC назначения'] = le_mac.transform(df['MAC назначения'])


scaler = MinMaxScaler()
df[['Time',
    'IP источника', 
    'ip назначения',
    'MAC источника',
    'MAC назначения']] = scaler.fit_transform(df[['Time',
                                                    'IP источника', 
                                                    'ip назначения',
                                                    'MAC источника',
                                                    'MAC назначения']])

Балансировка данных, можно рассмотреть как увелечение данных, рассмотерть подробнее

In [3]:
from sklearn.utils import resample

# Проверка баланса
print("Распределение классов:\n", df['Метка'].value_counts())

# Разделение на классы
normal_data = df[df['Метка'] == 0]
anomaly_data = df[df['Метка'] == 1]

# Увеличение меньшего класса (пример для случая, когда аномалий меньше)
anomaly_upsampled = resample(anomaly_data,
                             replace=True,
                             n_samples=len(normal_data),
                             random_state=42)

# Новый сбалансированный датасет
balanced_data = pd.concat([normal_data, anomaly_upsampled])


Распределение классов:
 Метка
0    2230
1     865
Name: count, dtype: int64


In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split



# Разделение на признаки и метки
X = df.drop('Метка', axis=1)
y = df['Метка']

# Разделение на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y

0      0
1      0
2      0
3      0
4      0
      ..
860    1
861    1
862    1
863    1
864    1
Name: Метка, Length: 3095, dtype: int64

In [5]:
import numpy as np

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def create_sequences(data, labels, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(labels[i+window_size])  # Метка для последнего элемента в окне
    return np.array(X), np.array(y)

window_size = 10  # Количество временных шагов в окне

# Преобразование данных в последовательности
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train.values, window_size)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test.values, window_size)




In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(64, input_shape=(window_size, X_train_seq.shape[2]), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Выходной слой

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', 'Precision', 'Recall', 'AUC']
)



d:\Проекты\Дипломаня работа\DoFitN\Code\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', save_best_only=True)  # Измени расширение!
]

history = model.fit(
    X_train_seq, y_train_seq,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=callbacks
)


Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1:22 1s/step - AUC: 0.6068 - Precision: 0.6000 - Recall: 0.3000 - accuracy: 0.7188 - loss: 0.67 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.5127 - Precision: 0.6000 - Recall: 0.0576 - accuracy: 0.7236 - loss: 0.6402 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.4877 - Precision: 0.6000 - Recall: 0.0313 - accuracy: 0.7153 - loss: 0.632 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.4835 - Precision: 0.6000 - Recall: 0.0222 - accuracy: 0.7166 - loss: 0.623 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - AUC: 0.4834 - Precision: 0.6000 - Recall: 0.0219 - accuracy: 0.7166 - loss: 0.6235 - val_AUC: 0.4812 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 0.7085 - val_loss: 0.6059
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - AUC: 0.5727 - Precision: 0.0000e+00 - Recall: 0.0000e+00 - accuracy: 0.6875 - loss: 0.61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.5249 - Precision: 0.0000e+00 - Recall: 0.0000e+00 - accuracy: 0.7161 - loss: 0.5970 ━━━━━━━━━━━━━━━